In [1]:
from aux.aux_functions import *

In [2]:
import openai
from openai import OpenAI

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

from IPython.display import display

In [4]:
from dotenv import load_dotenv
import os

load_dotenv("/Users/fmuia/Dropbox/NLP/Llama-FineTuning/config.env")
open_ai_key = os.getenv("OPENAI_API_KEY")

In [5]:
# Specify the path to your LaTeX file
latex_file_path = 'Files/2310.20559/main.tex'

# Extract the LaTeX content
latex_content = extract_latex_content(latex_file_path)

# For demonstration, let's print the first 500 characters to check if it worked
#print(latex_content[:15000])

In [10]:
simplified_text = replace_latex_commands(latex_content)
#simplified_text

In [11]:
polished_text = clean_latex_content(simplified_text)
#polished_text

In [12]:
no_char_in_chunk = 2000

chunks = chunk_text(polished_text, no_char_in_chunk)
chunks = [chunk.replace("%", "") for chunk in chunks]
print("The number of chunks is: " + str(len(chunks)) + ".")
#chunks[:2]

The number of chunks is: 143.


In [14]:
# Define the number of Q&A pairs you want to generate for each chunk
num_questions = 3

In [15]:
system_prompt_template = "You are an expert in theoretical physics. I will provide a text, and I'd like you to generate {num_questions} questions a researcher-level colleague might ask about the topic of the text. Then, provide detailed answers to these questions as an expeert in the field. Format your responses as follows: 'Question 1: [question here] Answer 1: [answer here]' and so on for all {num_questions} questions."

user_prompt_template = "Based on the following text, please generate the questions and answers as instructed. Text: {chunk}"

display(system_prompt_template)
display(user_prompt_template)


"You are an expert in theoretical physics. I will provide a text, and I'd like you to generate {num_questions} questions a researcher-level colleague might ask about the topic of the text. Then, provide detailed answers to these questions as an expeert in the field. Format your responses as follows: 'Question 1: [question here] Answer 1: [answer here]' and so on for all {num_questions} questions."

'Based on the following text, please generate the questions and answers as instructed. Text: {chunk}'

In [ ]:
client = OpenAI()

# Initialize a list to store chunk and response pairs
chunk_responses = []

# Assume chunks_test is a list of text chunks you want to generate Q&A for
for chunk in chunks:
    system_prompt, user_prompt = generate_prompt(chunk, num_questions, system_prompt_template, user_prompt_template)
    
    # Call the OpenAI API once per chunk
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[system_prompt, user_prompt]
    )
    
    # Append both the chunk and its response to the chunk_responses list
    chunk_responses.append({
        "chunk": chunk,
        "response": response.choices[-1].message.content if response.choices else "No response"
    })


In [41]:
# Initialize lists to hold the DataFrame data
chunk_list = []
question_list = []
answer_list = []
sub_index = []

# Process each chunk-response pair
for item in chunk_responses:
    chunk = item['chunk']
    response = item['response']
    
    # Split the response into Q&As
    qas = response.split('\n\n')  # Assuming two newlines separate each Q&A pair
    for i, qa in enumerate(qas):
        # Attempt to split the QA into question and answer
        parts = qa.split('\nAnswer ')
        if len(parts) == 2:
            question = parts[0].replace('Question {}: '.format(i+1), '').strip()
            answer = parts[1].strip()
            chunk_list.append(chunk)
            question_list.append(question)
            answer_list.append(answer)
            sub_index.append(i+1)

In [43]:
# Create a MultiIndex from the chunks and their corresponding sub_index
multi_index = pd.MultiIndex.from_arrays([chunk_list, sub_index], names=['Chunk', 'Q&A Index'])
answer_list = [re.sub(r'^\d+: ', '', answer) for answer in answer_list]

# Construct the DataFrame
df_multi = pd.DataFrame({'Question': question_list, 'Answer': answer_list}, index=multi_index)

# Display the DataFrame
df_multi

Question  \
Chunk                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Q&A Index                                                                                                                                               
Liam McAllister$^{a}$ and Fernando Quevedo$^{b}$ 30   We give an overview of moduli stabilization in compactifications of string theory. We summarize current methods for construction and analysis of vacua with stabilized moduli, and we describe applications to cosmology and particle physics. This is a contribution to the Handbook on Quantum Gravity.  The fundamental physical laws that govern our Universe must describe gravity and quantum mechanics. To discover the laws of quantum gravity, we cannot entirely rely on terrestrial experiments, or even on cosmological observations: the energies of observable processes are far too low to give a complete picture, in contrast to the way that collider experiments eventually revealed the Standard Model of particle physics. We may hope for some guidance from experiment, but theorists will have to provide a framework. String theory is such a framework: it is a theory of quantum gravity through which we can take a constructive approach to exploring possible laws of quantum gravity in our Universe. The first obstacle is that the world we observe at low energies is four-dimensional, while the best-understood solutions of string theory are ten-dimensional. Kaluza-Klein theory \cite{Kaluza:1921tu,Klein:1926tv}, now more than a century old, provides a way to bridge this gap. If the extra dimensions correspond to a six-dimensional compact space that is smaller than the reach of any experimental probe then only three spatial dimensions will be seen. However, the size and shape of the extra dimensions are dynamical: they are parameterized by the expectation values of scalar fields known as moduli. Unless the moduli have large masses, they mediate long-range forces that are not observed in our world. Thus, a central problem of Kaluza-Klein theories is to provide a dynamical explanation for the requisite size of 

In [45]:
qa_df = pd.DataFrame({
    'Question': question_list,
    'Answer': answer_list
})

qa_df

,Question,Answer
0,What are the current methods for constructing and analyzing vacua with stabilized moduli in compactifications of string theory?,"Stabilizing moduli in string theory compactifications involves finding solutions where the expectation values of scalar fields, known as moduli, are fixed at certain values to stabilize the extra dimensions. Current methods for constructing and analyzing such vacua include Flux Compactifications, Non-Perturbative Effects, and Gaugino Condensation, among others."
1,How does Kaluza-Klein theory bridge the gap between ten-dimensional string theory and the observed four-dimensional world we live in?,Kaluza-Klein theory proposes that the extra dimensions beyond the familiar four dimensions are compactified on a curled-up space known as the hidden dimensions. These extra dimensions are invisible because they are either too small to detect with current technology or because they are tightly curled up. The size and shape of these compact dimensions are dynamic and are described by scalar fields called moduli.
2,"Why is the stabilization of moduli necessary in string theory compactifications, and how does it relate to experimental observations?","Stabilizing the moduli in compactifications of string theory is crucial to ensuring that the extra dimensions are at an appropriate size and shape to match our observable universe. If the moduli fields are not stabilized or have masses that are too low, they could mediate long-range forces that have not been detected. Thus, ensuring the stabilization of moduli is essential to reconcile string theory with experimental observations and constraints from particle physics and cosmology."
3,What is the significance of a Calabi-Yau threefold in the context of string theory vacuum solutions?,"Calabi-Yau threefolds are special six-manifolds in string theory that have the property of being Ricci-flat with holonomy group contained in $SU(3)$. They are important in compactifying the extra dimensions of string theory, and finding vacuum solutions that preserve some supersymmetry, which aids in theoretical control."
4,Could there exist compact Ricci-flat six-manifolds with holonomy groups other than $SU(3)$ that furnish vacuum solutions in string theory?,"While there is no proof that $\text{Hol}(g) = SU(3)$ is a necessary condition for Ricci-flatness, all known examples of compact Ricci-flat six-manifolds have holonomy contained in $SU(3).$ However, it is theoretically possible that there are compact six-manifolds with holonomy $SO(6)$ that could lead to non-supersymmetric vacua in string theory."
5,How does the presence of supersymmetry in Calabi-Yau compactifications relate to the hierarchy problem in particle physics?,"The presence of supersymmetry in Calabi-Yau compactifications of string theory is not directly related to solving the hierarchy problem in particle physics. Instead, it provides a powerful tool for theoretical control. Superstring theories, which are supersymmetric, are better understood compared to non-supersymmetric string theories. Calabi-Yau compactifications preserve some supersymmetry at the compactification scale, which may be broken at lower energies."
